## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from keras.callbacks import ModelCheckpoint

In [2]:
#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=["EIN","NAME"])

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [4]:
# Determine the number of unique values in each column.
application_df.nunique()

EIN                       34299
NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [5]:
# Look at APPLICATION_TYPE value counts for binning
application_type_counts = application_df["APPLICATION_TYPE"].value_counts()

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list( application_type_counts[ application_type_counts < 500].index)
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
application_classification_counts = application_df["CLASSIFICATION"].value_counts()
application_classification_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_classification_gt_count = application_classification_counts > 1
application_classification_gt_count

C1000     True
C2000     True
C1200     True
C3000     True
C2100     True
         ...  
C4120    False
C8210    False
C2561    False
C4500    False
C2150    False
Name: CLASSIFICATION, Length: 71, dtype: bool

In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(application_classification_counts[ application_classification_counts < 15000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
Other    16973
Name: CLASSIFICATION, dtype: int64

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
hot_df = pd.get_dummies(application_df[["AFFILIATION","CLASSIFICATION","USE_CASE","ORGANIZATION","STATUS","INCOME_AMT","SPECIAL_CONSIDERATIONS","IS_SUCCESSFUL"]])

In [11]:
application_model_df = pd.concat([hot_df,application_df["ASK_AMT"]],axis=1)
application_model_df

,STATUS,IS_SUCCESSFUL,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,CLASSIFICATION_C1000,CLASSIFICATION_Other,...,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,ASK_AMT
0,1,1,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,5000
1,1,1,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,108590
2,1,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,5000
3,1,1,1,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,1,0,6692
4,1,1,0,0,1,0,0,0,1,0,...,0,1,0,0,0,0,0,1,0,142590
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,5000
34295,1,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,5000
34296,1,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,5000
34297,1,1,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,5000


In [12]:
# Split our preprocessed data into our features and target arrays
y = application_model_df["IS_SUCCESSFUL"]
# Split the preprocessed data into a training and testing dataset
X = application_model_df.drop(["IS_SUCCESSFUL"],1)
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42)

<ipython-input-12-ca6d8e1c4bd0>:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = application_model_df.drop(["IS_SUCCESSFUL"],1)


In [13]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
application_input_features = len(X_train)
hidden_nodes_layer1 =  90
hidden_nodes_layer3 = 45
hidden_nodes_layer2 = 20
application_nn_optimized = tf.keras.models.Sequential()

# First hidden layer
application_nn_optimized.add(
    tf.keras.layers.InputLayer(input_shape=(30))
)

# Second hidden layer
application_nn_optimized.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

#third hidden layer
# application_nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))


# Output layer
application_nn_optimized.add(tf.keras.layers.Dense(units=1, activation="ReLU"))

# Check the structure of the model
application_nn_optimized.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 20)                620       
                                                                 
 dense_1 (Dense)             (None, 1)                 21        
                                                                 
Total params: 641
Trainable params: 641
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Compile the model
application_nn_optimized.compile(loss="categorical_crossentropy", optimizer="adam", metrics="accuracy")

In [16]:
checkpoint_path = 'checkpoints.hdf5'

In [17]:
#create checkpoints
checkpoint = ModelCheckpoint(filepath=checkpoint_path,
                             save_freq = 'epoch',
                             monitor='val_accuracy',
                             verbose=1,
                             mode='max')

In [19]:
# Train the model and save weights for every 5 epochs
application_nn_optimized.fit(X_train_scaled, y_train, epochs=25, callbacks=[checkpoint])

Epoch 1/25


/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/dispatch.py:1176: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


796/804 [============================>.] - ETA: 0s - loss: nan - accuracy: 0.4682
Epoch 1: saving model to checkpoints.hdf5
804/804 [==============================] - 2s 1ms/step - loss: nan - accuracy: 0.4682
Epoch 2/25
802/804 [============================>.] - ETA: 0s - loss: nan - accuracy: 0.4684
Epoch 2: saving model to checkpoints.hdf5
804/804 [==============================] - 1s 1ms/step - loss: nan - accuracy: 0.4682
Epoch 3/25
799/804 [============================>.] - ETA: 0s - loss: nan - accuracy: 0.4683
Epoch 3: saving model to checkpoints.hdf5
804/804 [==============================] - 1s 1ms/step - loss: nan - accuracy: 0.4682
Epoch 4/25
791/804 [============================>.] - ETA: 0s - loss: nan - accuracy: 0.4684
Epoch 4: saving model to checkpoints.hdf5
804/804 [==============================] - 1s 1ms/step - loss: nan - accuracy: 0.4682
Epoch 5/25
798/804 [============================>.] - ETA: 0s - loss: nan - accuracy: 0.4681
Epoch 5: saving model to checkpoin

In [22]:
# Evaluate the model using the test data
model_loss, model_accuracy = application_nn_optimized.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/dispatch.py:1176: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


268/268 - 1s - loss: nan - accuracy: 0.4657 - 879ms/epoch - 3ms/step
Loss: nan, Accuracy: 0.46565598249435425


In [23]:
# Export our model to HDF5 file
application_nn_optimized.save('AlphabetSoupCharity_optimized.h5')

FileNotFoundError: ignored